In [29]:
import pandas as pd
from datetime import datetime

In [30]:
import os
os.getcwd()


'C:\\Users\\Vaishali S\\anaconda3'

In [32]:
df = pd.read_excel(
   "Fashion_Retail_Sales (1).xlsx",
)


In [33]:
df.head()

,Customer Reference ID,Item Purchased,Purchase Amount (USD),Date Purchase,Review Rating,Payment Method
0,4018,Handbag,4619.0,2023-02-05,NaN,Credit Card
1,4115,Tunic,2456.0,2023-07-11,2.0,Credit Card
2,4019,Tank Top,2102.0,2023-03-23,4.1,Cash
3,4097,Leggings,3126.0,2023-03-15,3.2,Cash
4,3997,Wallet,3003.0,2022-11-27,4.7,Cash


In [34]:
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Customer Reference ID  3400 non-null   int64         
 1   Item Purchased         3400 non-null   object        
 2   Purchase Amount (USD)  2750 non-null   float64       
 3   Date Purchase          3400 non-null   datetime64[ns]
 4   Review Rating          3076 non-null   float64       
 5   Payment Method         3400 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 159.5+ KB


Customer Reference ID      0
Item Purchased             0
Purchase Amount (USD)    650
Date Purchase              0
Review Rating            324
Payment Method             0
dtype: int64

In [37]:
df['Date Purchase'] = pd.to_datetime(df['Date Purchase'])


In [12]:
analysis_date = df['Date Purchase'].max() + pd.Timedelta(days=1)
analysis_date


Timestamp('2025-01-01 00:00:00')

In [38]:
rfm = df.groupby('Customer Reference ID').agg(
    Recency=('Date Purchase', lambda x: (analysis_date - x.max()).days),
   Frequency=('Date Purchase', 'count'),
    Monetary=('Purchase Amount (USD)', 'sum')
).reset_index()


In [39]:
rfm.columns = ['Customer Reference ID', 'Recency', 'Frequency', 'Monetary']
rfm.head()


,Customer Reference ID,Recency,Frequency,Monetary
0,3957,543,14,1150.0
1,3958,475,20,1334.0
2,3959,486,22,2047.0
3,3960,475,18,1434.0
4,3961,491,22,1811.0


In [40]:
rfm.describe()


,Customer Reference ID,Recency,Frequency,Monetary
count,166.000000,166.000000,166.000000,166.000000
mean,4039.500000,476.728916,20.481928,2596.096386
std,48.064193,19.428626,4.375861,1792.654863
min,3957.000000,458.000000,10.000000,802.000000
25%,3998.250000,464.000000,17.000000,1486.250000
50%,4039.500000,470.000000,20.500000,1905.500000
75%,4080.750000,484.750000,23.000000,2815.250000
max,4122.000000,574.000000,34.000000,9685.000000


In [41]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])


In [46]:
rfm['RFM_Score'] = (
    rfm['R_Score'].astype(str) +
    rfm['F_Score'].astype(str) +
    rfm['M_Score'].astype(str)
)

rfm.head()


,Customer Reference ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,3957,543,14,1150.0,1,1,1,111
1,3958,475,20,1334.0,3,3,1,331
2,3959,486,22,2047.0,2,3,3,233
3,3960,475,18,1434.0,3,2,2,322
4,3961,491,22,1811.0,1,3,3,133


In [50]:
rfm.columns = rfm.columns.str.replace(' ', '_')

In [52]:
rfm[rfm['Customer_Reference_ID'] == 3959]

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
2,3959,486,22,2047.0,2,3,3,233


In [48]:
rfm[rfm.RFM_Score== '555'] #Best customers

,Customer Reference ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
83,4040,462,25,9657.0,5,5,5,555
151,4108,458,26,6864.0,5,5,5,555
165,4122,460,32,5652.0,5,5,5,555


Who are our highest-value customers right now?

In [55]:
rfm[(rfm.RFM_Score== '555') | (rfm.RFM_Score== '554') | (rfm.RFM_Score== '545') | (rfm.RFM_Score== '544') | (rfm.RFM_Score== '455') |(rfm.RFM_Score== '454') |(rfm.RFM_Score== '445') |(rfm.RFM_Score== '444') ] 

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
13,3970,463,27,2341.0,4,5,4,454
21,3978,461,27,3110.0,5,5,4,554
28,3985,465,25,2076.0,4,5,4,454
29,3986,464,23,6097.0,4,4,5,445
31,3988,465,25,2202.0,4,5,4,454
35,3992,467,24,2238.0,4,4,4,444
60,4017,461,24,5012.0,5,4,5,545
83,4040,462,25,9657.0,5,5,5,555
92,4049,461,22,2069.0,5,4,4,544
94,4051,465,26,2093.0,4,5,4,454


Are we acquiring customers who become valuable, or just one-time buyers?

In [72]:
rfm[(rfm.Frequency == 1)]

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score


In [68]:
rfm[(rfm.Frequency <11)]

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
155,4112,488,10,885.0,2,1,1,211


The absence of one-time buyers and consistently high purchase frequency (≥10 purchases per customer) indicate strong product-market fit and high customer satisfaction.

Which valuable customers are at risk of churn?

In [70]:
rfm[(rfm.RFM_Score== '155') | (rfm.RFM_Score== '154') | (rfm.RFM_Score== '145') | (rfm.RFM_Score== '244') | (rfm.RFM_Score== '255') |(rfm.RFM_Score== '254') ] 

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
16,3973,485,29,5092.0,2,5,5,255
32,3989,476,22,2134.0,2,4,4,244
37,3994,479,25,2201.0,2,5,4,254
45,4002,492,26,6320.0,1,5,5,155
51,4008,482,27,2094.0,2,5,4,254
77,4034,482,32,2524.0,2,5,4,254
85,4042,486,22,2221.0,2,4,4,244
87,4044,496,24,8745.0,1,4,5,145
113,4070,484,28,4964.0,2,5,5,255
120,4077,486,22,2374.0,2,4,4,244


Historically high-value customers (high Frequency / Monetary) have been inactive for 470+ days, highlighting a critical reactivation opportunity.

Customers who want to be avoided in paid retargeting

In [71]:
rfm[(rfm.RFM_Score== '111') | (rfm.RFM_Score== '121') | (rfm.RFM_Score== '122')] 

,Customer_Reference_ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,3957,543,14,1150.0,1,1,1,111
14,3971,491,18,1493.0,1,2,2,122
56,4013,498,18,1463.0,1,2,2,122
65,4022,493,13,1254.0,1,1,1,111
67,4024,494,15,1375.0,1,1,1,111
73,4030,521,16,1236.0,1,1,1,111
81,4038,506,19,1448.0,1,2,2,122
82,4039,519,16,1093.0,1,1,1,111
93,4050,525,14,1004.0,1,1,1,111
104,4061,521,18,1284.0,1,2,1,121
